In [1]:
import numpy as np
import h5py
import random

In [3]:
f = h5py.File("/home/g202321530/Yang/Data/Earthquake_data/TXED/TXED_0913.h5", 'r')
# eventid = np.load('/home/g202321530/Yang/Data/Earthquake_data/TXED/ID_0913.npy')
allid = np.load("/home/g202321530/Yang/Data/Earthquake_data/TXED/ID_0913.npy")
noiseid=[ii for ii in allid if ii.split("_")[-1]=='NO']
signalid=[ii for ii in allid if ii.split("_")[-1]=='EV']
# print(eventid.shape)

# signalid = np.load('/home/g202321530/Yang/Data/Earthquake_data/TXED/txed_signal_id.npy')
# print('shape of signal ID:', signalid.shape)

In [4]:
# signalid1_2 = signalid[110000: 160000]
random.seed(202410)
signal_random = random.sample(signalid, 100000) 
signalid1_2 = signal_random
print(signal_random[:10])

P_arrival_list = []
S_arrival_list = []
print('-----------signal format convert begin-------------------')
for key in signalid1_2:
    if key in f:   
        dataset = f.get(key)
        P_arrival_list.append(int(dataset.attrs['p_arrival_sample']))
        S_arrival_list.append(int(dataset.attrs['s_arrival_sample']))
P_arrival_list = np.array(P_arrival_list)
S_arrival_list = np.array(S_arrival_list)
print(P_arrival_list.shape, P_arrival_list)
print(S_arrival_list.shape, S_arrival_list)
np.save('./data/arrival_time/Pwave_arrival_txed_signal_random_10w.npy', P_arrival_list)
np.save('./data/arrival_time/Swave_arrival_txed_signal_random_10w.npy', S_arrival_list)
print('-----------signal format convert end-------------------')

['texnet2022sdyz_PB29_EV', 'texnet2022zmjp_PB38_EV', 'texnet2022yieq_SA06_EV', 'texnet2020juyx_MB02_EV', 'texnet2023clhe_APMT_EV', 'texnet2022hpvu_PB12_EV', 'texnet2022uowl_SA09_EV', 'texnet2022xgwl_SA02_EV', 'texnet2018ohkk_FW16_EV', 'texnet2020yand_ALPN_EV']
-----------signal format convert begin-------------------
(100000,) [ 99 498  98 ... 599 499 299]
(100000,) [ 616  709  382 ...  781  693 1627]
-----------signal format convert end-------------------


In [5]:
import numpy as np
import matplotlib.pyplot as plt

def generate_first_arrival_labels(p_indx, s_indx, sample_num):
    """
    Generate labels for P-wave and S-wave first arrival picking.

    Parameters:
        p_indx (float): Arrival time of P-wave.
        s_indx (float): Arrival time of S-wave.
        sample_num (float): Total duration of the seismic signal.

    Returns:
        np.array: Array of labels indicating P-wave, S-wave, or neither.
    """

    # Initialize labels array
    labels = np.zeros(sample_num)
    
    # Set labels for P-wave and S-wave first arrivals
    labels[:p_indx] = 1  # P-wave
    labels[p_indx:s_indx] = 3  # S-wave
    
    return labels


def generate_first_arrival_labels_1(p_indx, s_indx, sample_num):
    """
    Generate labels for P-wave and S-wave first arrival picking.

    Parameters:
        p_indx (float): Arrival time of P-wave.
        s_indx (float): Arrival time of S-wave.
        sample_num (float): Total duration of the seismic signal.

    Returns:
        np.array: Array of labels indicating P-wave, S-wave, or neither.
    """
    all_time = np.linspace(0, sample_num, sample_num)
    #print(sample_num, all_time.shape)
    p_wave_arrival_time = p_indx  # P-wave arrival time (in seconds)
    s_wave_arrival_time = s_indx  # S-wave arrival time (in seconds)
    
    # Create Gaussian-shaped labels for P-wave and S-wave arrivals
    sigma = 10 # Standard deviation of the Gaussian
    p_wave_label = np.exp(-(all_time - p_wave_arrival_time)**2 / (2 * sigma**2))  # Gaussian label for P-wave
    s_wave_label = np.exp(-(all_time - s_wave_arrival_time)**2 / (2 * sigma**2))  # Gaussian label for S-wave

    return p_wave_label, s_wave_label



def visualize_input_and_labels(data, labels, spt, sst):
    """
    Visualize seismic signal along with labels indicating P-wave and S-wave first arrival picking.

    Parameters:
        signal (np.array): Seismic signal.
        labels (np.array): Array of labels indicating P-wave, S-wave, or neither.
        sampling_rate (float): Sampling rate of the seismic signal.
    """
    
    # Plot the seismic signal
    fig = plt.figure(figsize=(10, 6))
    ax1 = fig.add_subplot(311)
    plt.plot(data[:,0], 'k',label='Z')
    ymin,yma = ax1.get_ylim()
    plt.vlines(spt,ymin,yma,color='r',linewidth=2)
    plt.vlines(sst,ymin,yma,color='b',linewidth=2)
    legend_properties = {'weight':'bold'}
    ymin, ymax = ax1.get_ylim()
    plt.legend(loc = 'upper right', borderaxespad=0., prop=legend_properties)
    plt.ylabel('Amplitude', fontsize=12)
    plt.xlim([0, 6000])
    ax1.set_xticklabels([])
    ax = fig.add_subplot(312) 
    plt.plot(data[:,1], 'k',label='N')
    ymin,yma = ax1.get_ylim()
    plt.vlines(spt,ymin,yma,color='r',linewidth=2)
    plt.vlines(sst,ymin,yma,color='b',linewidth=2)
    legend_properties = {'weight':'bold'}
    ymin, ymax = ax.get_ylim()
    plt.legend(loc = 'upper right', borderaxespad=0., prop=legend_properties)
    plt.ylabel('Amplitude', fontsize=12) 
    plt.xlim([0, 6000])
    ax.set_xticklabels([])
    ax = fig.add_subplot(313) 
    plt.plot(data[:,2], 'k',label='E')
    ymin,yma = ax1.get_ylim()
    plt.vlines(spt,ymin,yma,color='r',linewidth=2)
    plt.vlines(sst,ymin,yma,color='b',linewidth=2)
    legend_properties = {'weight':'bold'}
    ymin, ymax = ax.get_ylim()
    plt.legend(loc = 'upper right', borderaxespad=0., prop=legend_properties)
    plt.ylabel('Amplitude', fontsize=12) 
    plt.xlabel('Sample', fontsize=12) 
    plt.xlim([0, 6000])
    ymin,yma = ax.get_ylim()
    

    
    # Plot vertical lines indicating P-wave and S-wave first arrival picking
    p_indices = np.where(labels == 1)[0]
    s_indices = np.where(labels == 3)[0]
    
    # if len(p_indices) > 0:
    #     p_time = p_indices[-1]
    #     print('P_time:', p_time)
    #     plt.vlines(p_time, ymin, yma, color='red', linestyle='--', label='P-wave Arrival')
    # if len(s_indices) > 0:
    #     s_time = s_indices[-1]
    #     print('s_time:', s_time)
    #     plt.vlines(s_time, ymin, yma, color='blue', linestyle='--', label='S-wave Arrival')
    
    # Add labels and legend
    # plt.xlabel('Time (s)')
    # plt.ylabel('Amplitude')
    # plt.title('Seismic Signal with P-wave and S-wave First Arrival Picking')
    # plt.legend()
    # plt.grid(True)
    
    # Show the plot
    # plt.show()

def plot_labels_signals(signal, p_labels, s_labels, sample_num):
    data = signal
    time = np.linspace(0, sample_num, sample_num)
    fig = plt.figure(figsize=(10, 6))
    ax1 = fig.add_subplot(411)
    plt.plot(time, data[:,0], 'k',label='Z')
    ymin,yma = ax1.get_ylim()
    # plt.vlines(spt,ymin,yma,color='r',linewidth=2)
    # plt.vlines(sst,ymin,yma,color='b',linewidth=2)
    legend_properties = {'weight':'bold'}
    ymin, ymax = ax1.get_ylim()
    plt.legend(loc = 'upper right', borderaxespad=0., prop=legend_properties)
    plt.ylabel('Amplitude', fontsize=12)
    plt.xlim([0, 6000])
    ax1.set_xticklabels([])
    ax = fig.add_subplot(412) 
    plt.plot(time, data[:,1], 'k',label='N')
    ymin,yma = ax1.get_ylim()
    # plt.vlines(spt,ymin,yma,color='r',linewidth=2)
    # plt.vlines(sst,ymin,yma,color='b',linewidth=2)
    legend_properties = {'weight':'bold'}
    ymin, ymax = ax.get_ylim()
    plt.legend(loc = 'upper right', borderaxespad=0., prop=legend_properties)
    plt.ylabel('Amplitude', fontsize=12) 
    plt.xlim([0, 6000])
    ax.set_xticklabels([])
    ax = fig.add_subplot(413) 
    plt.plot(time, data[:,2], 'k',label='E')
    ymin,yma = ax1.get_ylim()
    # plt.vlines(spt,ymin,yma,color='r',linewidth=2)
    # plt.vlines(sst,ymin,yma,color='b',linewidth=2)
    legend_properties = {'weight':'bold'}
    ymin, ymax = ax.get_ylim()
    plt.legend(loc = 'upper right', borderaxespad=0., prop=legend_properties)
    plt.ylabel('Amplitude', fontsize=12) 
    plt.xlim([0, 6000])
    ymin,yma = ax.get_ylim()
    ax = fig.add_subplot(414) 
    plt.plot(time, p_labels, label='P-wave Arrival', linestyle='--')
    plt.plot(time, s_labels, label='S-wave Arrival', linestyle='--')
    plt.legend(loc = 'upper right', borderaxespad=0., prop=legend_properties)
    plt.ylabel('Probability', fontsize=12) 
    plt.xlabel('Sample', fontsize=12) 
    plt.xlim([0, 6000])
    plt.tight_layout()
    
    # plt.figure(figsize=(10, 6))
    # plt.plot(time, signal, label='Seismic Waveform')
    # plt.plot(time, p_labels, label='P-wave Arrival', linestyle='--')
    # plt.plot(time, s_labels, label='S-wave Arrival', linestyle='--')
    # plt.xlabel('Time (s)')
    # plt.ylabel('Amplitude')
    # plt.title('Synthetic Seismic Waveform with Gaussian-shaped Labels')
    # plt.legend()
    # plt.grid(True)
    # plt.show()

### The first label generation method

In [6]:
signal_all = np.load('./data/signal_events/txed_signal11_16_bp.npy')
signals = signal_all[:50, :, :]
labels_all = []
for i in range (signals.shape[0]):
    # Example usage:
    p_indx = P_arrival_list[i]  # Arrival time of P-wave (in seconds)
    s_indx = S_arrival_list[i]  # Arrival time of S-wave (in seconds)    
    sample_num = signal_all.shape[1]  # Sampling rate of seismic signal (in Hz)
    print(f'Processed_num:{i}\t P_arrival_time: {p_indx}\t S_arrival_time: {s_indx}\t sample_num: {sample_num}')
    
    # Generate labels for P-wave and S-wave first arrival picking
    labels = generate_first_arrival_labels(p_indx, s_indx, sample_num)
    labels_all.append(labels)    
    signal = signals[i, :, :]
    
    # Visualize the seismic signal along with the labels
    # visualize_input_and_labels(signal, labels, spt=p_indx, sst=s_indx)
    # # plt.close()
    # plt.show()
print((np.array(labels_all)).shape)
# np.save('./data/arrival_time/picking_labels_txed_signal15_16.npy', labels_all)

Processed_num:0	 P_arrival_time: 99	 S_arrival_time: 616	 sample_num: 6000
Processed_num:1	 P_arrival_time: 498	 S_arrival_time: 709	 sample_num: 6000
Processed_num:2	 P_arrival_time: 98	 S_arrival_time: 382	 sample_num: 6000
Processed_num:3	 P_arrival_time: 99	 S_arrival_time: 2755	 sample_num: 6000
Processed_num:4	 P_arrival_time: 299	 S_arrival_time: 1291	 sample_num: 6000
Processed_num:5	 P_arrival_time: 299	 S_arrival_time: 1015	 sample_num: 6000
Processed_num:6	 P_arrival_time: 598	 S_arrival_time: 1056	 sample_num: 6000
Processed_num:7	 P_arrival_time: 499	 S_arrival_time: 797	 sample_num: 6000
Processed_num:8	 P_arrival_time: 499	 S_arrival_time: 1248	 sample_num: 6000
Processed_num:9	 P_arrival_time: 299	 S_arrival_time: 2199	 sample_num: 6000
Processed_num:10	 P_arrival_time: 299	 S_arrival_time: 657	 sample_num: 6000
Processed_num:11	 P_arrival_time: 398	 S_arrival_time: 2632	 sample_num: 6000
Processed_num:12	 P_arrival_time: 698	 S_arrival_time: 1197	 sample_num: 6000
Proc

### The second label generation method

In [9]:
# signals_1 = signal_all[:, :, :]
from scipy.signal import butter, lfilter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

from scipy.signal import butter, lfilter, lfilter_zi

def butter_bandpass_filter_zi(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    zi = lfilter_zi(b, a)
    y,zo = lfilter(b, a, data, zi=zi*data[0])
    return y
    
signal_list = []
print('-----------signal format convert begin-------------------')
for key in signalid1_2:
    if key in f:   
        dataset = f.get(key)
        datas = dataset['data']
        datas = np.array(datas)
        datas_0 = butter_bandpass_filter_zi(datas[:,0], 1, 45, 100, order=3)
        datas_1 = butter_bandpass_filter_zi(datas[:,1], 1, 45, 100, order=3)
        datas_2 = butter_bandpass_filter_zi(datas[:,2], 1, 45, 100, order=3)
        datas = np.vstack([datas_0, datas_1, datas_2])
        signal_list.append(datas) 
signal_values = np.array(signal_list)
signal_values = np.transpose(signal_values, [0, 2, 1])

np.save('./data/signal_events/random_10w_1006_bp.npy', signal_values)
signals_1 = signal_values
p_wave_label = []
s_wave_label = []

for i in range (signals_1.shape[0]):
    # Example usage:
    p_indx = P_arrival_list[i]  # Arrival time of P-wave (in seconds)
    s_indx = S_arrival_list[i]  # Arrival time of S-wave (in seconds)    
    sample_num = signal_all.shape[1]  # Sampling rate of seismic signal (in Hz)
    #print(f'Processed_num:{i}\t P_arrival_time: {p_indx}\t S_arrival_time: {s_indx}\t sample_num: {sample_num}')
    if i % 1000 == 0:
        print(f'Processed_num: {i}\t P_arrival_time: {p_indx}\t S_arrival_time: {s_indx}\t sample_num: {sample_num}')
    
    # Generate labels for P-wave and S-wave first arrival picking
    p_labels, s_labels = generate_first_arrival_labels_1(p_indx, s_indx, sample_num)
    # print('shape:', p_labels.shape, s_labels.shape, p_labels, s_labels)
    p_wave_label.append(p_labels)  
    s_wave_label.append(s_labels)  
    signal = signals_1[i, :, :]
    
    # Visualize the seismic signal along with the labels
    # visualize_input_and_labels(signal, labels, spt=p_indx, sst=s_indx)
    # plot_labels_signals(signal, p_labels, s_labels, sample_num)
    # # plt.close()
    # plt.show()
p_wave_label = np.array(p_wave_label)
p_wave_label = np.reshape(p_wave_label, [p_wave_label.shape[0], p_wave_label.shape[1], 1])
s_wave_label = np.array(s_wave_label)
s_wave_label = np.reshape(s_wave_label, [s_wave_label.shape[0], s_wave_label.shape[1], 1])
label_all = np.concatenate([p_wave_label, s_wave_label], axis=-1)
print(f'shape_of_p_labels: {p_wave_label.shape}\t shape_of_s_labels: {s_wave_label.shape}\t shape_of_all_labels: {label_all.shape}')
np.save('./data/arrival_time/guassian_picking_labels_Pwave_txed_signal_random_10w.npy', p_wave_label)
np.save('./data/arrival_time/guassian_picking_labels_Swave_txed_signal_random_10w.npy', s_wave_label)
np.save('./data/arrival_time/guassian_picking_labels_all_txed_signal_random_10w.npy', label_all)

Processed_num: 0	 P_arrival_time: 99	 S_arrival_time: 616	 sample_num: 6000
Processed_num: 1000	 P_arrival_time: 898	 S_arrival_time: 2194	 sample_num: 6000
Processed_num: 2000	 P_arrival_time: 399	 S_arrival_time: 772	 sample_num: 6000
Processed_num: 3000	 P_arrival_time: 698	 S_arrival_time: 2632	 sample_num: 6000
Processed_num: 4000	 P_arrival_time: 798	 S_arrival_time: 1032	 sample_num: 6000
Processed_num: 5000	 P_arrival_time: 598	 S_arrival_time: 741	 sample_num: 6000
Processed_num: 6000	 P_arrival_time: 799	 S_arrival_time: 1205	 sample_num: 6000
Processed_num: 7000	 P_arrival_time: 599	 S_arrival_time: 3185	 sample_num: 6000
Processed_num: 8000	 P_arrival_time: 899	 S_arrival_time: 1473	 sample_num: 6000
Processed_num: 9000	 P_arrival_time: 198	 S_arrival_time: 554	 sample_num: 6000
Processed_num: 10000	 P_arrival_time: 98	 S_arrival_time: 328	 sample_num: 6000
Processed_num: 11000	 P_arrival_time: 898	 S_arrival_time: 1625	 sample_num: 6000
Processed_num: 12000	 P_arrival_time

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate random data
data = np.random.randn(1000)

# Create histogram
plt.figure(figsize=(10, 6))  # Set figure size
plt.hist(data, bins=10, color='skyblue', edgecolor='black', alpha=0.7)  # Adjust bins, color, edgecolor, and transparency
plt.title('Histogram of Random Data', fontsize=16, fontweight='bold')  # Add title with custom font size and weight
plt.xlabel('Values', fontsize=12)  # Add x-axis label with custom font size
plt.ylabel('Frequency', fontsize=12)  # Add y-axis label with custom font size
plt.xticks(fontsize=10)  # Set font size for x-axis ticks
plt.yticks(fontsize=10)  # Set font size for y-axis ticks
plt.grid(axis='y', linestyle='--', alpha=0.5)  # Add grid lines with custom style and transparency
plt.show()